# The CHSH Game

In [1]:
%matplotlib inline

from qiskit import *
from qiskit.tools.monitor import *
from qiskit.providers.ibmq import least_busy
import numpy as np

## A game to test whether IBM's  quantum computers are truly quantum

### We create a function that simulates the same Alice and Bob game 
<li> Alice does the measurement on angles a0 and a1
    <li> Bob does the same on b0 and b1
        <li> x, y are the input bits that are sent by a refree

In [2]:
def CHSH_circuit(x,y, a0 = 0, a1 = np.pi/2, b0= np.pi/4, b1 = -np.pi/4):
    circ = QuantumCircuit(2,2)
    
    #making the bell paiir
    circ.h(0)
    circ.cx(0,1)
    
    
    #x is recieved by Alice 
    #y is recieved by Bob
    #if the recieved bit is 0 alice applies a0
    #if the recieved bit is 1 alice applies a1
    #if the recieved  bit by bob is 0 bob applies b0
    #if the recieved bit by bob is 1 bob applies b1
    
    if(x==0):  #input bit is 0
        circ.ry(a0,0)   #applying the Ry rotation by an angle a0=0(leaving the state as it is)
    else:     #input bit is 1
        circ.ry(a1,0)  #applying the Ryrotation by an angle of pi/2
        
    if(y==0): #input bit is 0
        circ.ry(b0,1)
    else:   #input bit is 1
        circ.ry(b1,1)
        
        
    circ.measure(range(2), range(2))
    
    return circ

### Computing the Winning Probability

In [3]:
def winning_probability(backend, shots = 8192, a0 = 0, a1 = np.pi/2, b0 =np.pi/4, b1 = -np.pi/4):
    total = 0  #here the probability will be saved dividing it with 4*(number of shots))
    circuits = [CHSH_circuit(0,0,a0,a1,b0,b1), CHSH_circuit(0,1,a0,a1,b0,b1), CHSH_circuit(1,0,a0,a1,b0,b1), CHSH_circuit(1,1,a0,a1,b0,b1)]
    ## applying all the four possibilities of the inputs, |00> , |01>, |10> and |11>
    ## and applying their respective angle measurement by alice and bob
    
    
    job = execute(circuits, backend = backend, shots = shots)
    
    #defining the job function where they execute the all 4 circuits
    #entering the number of shots to be 8192
    #looping the full circuit to run 4 times for all the four circuits we have
    #applying the job and getting the result and storing the counts
    #checking the counts
    for qc in circuits[0:3]:
        counts = job.result().get_counts(qc)
        if('00' in counts): #checking whether the result is |00>
            total += counts['00'] # if the result is |00> then increase the total count by number of |00>
        if('11' in counts): #checking whether the result is |11>
            total += counts['11'] # if the result is |11> then increase the total count by the number of |11>
    # For the fourth circuit, Alice's and Bob's outputs must be different for them to win
    counts = job.result().get_counts(circuits[3])
    if('01' in counts):
        total += counts['01']
    if('10' in counts):
        total += counts['10']
        
    return total/(4*shots)  
    

## Doing it on the qasm simulator

In [4]:
# Backend definition 

backend = Aer.get_backend('qasm_simulator')

# Execution

print(winning_probability(backend))

0.850372314453125


## Doing it On the Quantum Computer

In [6]:
# Load the account

provider = IBMQ.load_account()

# We execute on the least busy device (among the actual quantum computers)

backend = least_busy(provider.backends(operational = True, simulator=False, status_msg='active',
                                       filters=lambda x: x.configuration().n_qubits > 1)) 
print("We are executing on...",backend)

print(winning_probability(backend))

ibmqfactory.load_account:WARNING:2020-12-22 16:28:50,366: Credentials are already in use. The existing account in the session will be replaced.


We are executing on... ibmq_vigo
0.790435791015625
